# Downloading Experiments

This notebook is a proof of concept

Right now, we have some flaws in our raw data download system. We want to make sure we get this right because we will be downloading and processing hundreds or thousands of GBs of raw data parquet files.

So, we'll be messing around here with some implementations, and if they work, we'll be replacing parts of our original code with this.

In [1]:
%pip install huggingface_hub --quiet


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Getting file names

We want to use file names and other meta data to do dupe checks of what we've already processed, and see what we still need to download from a certain month. Let's run this code that gets all file names from the repo, to see what the data looks like.

In [2]:
# Here we get a list of raw data files for the given month/year

# File names in the remote repo are structured like:
# data/year=2025/month=03/train-00001-of-00065.parquet
# Obviously, there will be different amounts of them so it won't always be -00065.parquet

from huggingface_hub import HfApi
from pathlib import Path

year = "2025"
month = "03"  # <-- type manually

api = HfApi()
files = api.list_repo_files(
    repo_id="Lichess/standard-chess-games",
    repo_type="dataset"
)

# Filter for that year/month
target_prefix = f"data/year={year}/month={month}/"
all_file_names_in_month = [f for f in files if f.startswith(target_prefix)]

print(len(all_file_names_in_month), f"files found for {year}-{month}")
for f in all_file_names_in_month[:20]:  # preview first 20
    print(f)

69 files found for 2025-03
data/year=2025/month=03/train-00000-of-00069.parquet
data/year=2025/month=03/train-00001-of-00069.parquet
data/year=2025/month=03/train-00002-of-00069.parquet
data/year=2025/month=03/train-00003-of-00069.parquet
data/year=2025/month=03/train-00004-of-00069.parquet
data/year=2025/month=03/train-00005-of-00069.parquet
data/year=2025/month=03/train-00006-of-00069.parquet
data/year=2025/month=03/train-00007-of-00069.parquet
data/year=2025/month=03/train-00008-of-00069.parquet
data/year=2025/month=03/train-00009-of-00069.parquet
data/year=2025/month=03/train-00010-of-00069.parquet
data/year=2025/month=03/train-00011-of-00069.parquet
data/year=2025/month=03/train-00012-of-00069.parquet
data/year=2025/month=03/train-00013-of-00069.parquet
data/year=2025/month=03/train-00014-of-00069.parquet
data/year=2025/month=03/train-00015-of-00069.parquet
data/year=2025/month=03/train-00016-of-00069.parquet
data/year=2025/month=03/train-00017-of-00069.parquet
data/year=2025/mont

## Dupe checks

Now that we have the list of raw data file names for the given month and year, we'll perform our dupe checks. This parses through the list of file names to make sure we haven't already processed any of these files.

Each file is a 1GB download, so it's obviously in our best interest not to download a file we've already processed.

In [3]:
# Parse the list of raw data file names to make sure we haven't already processed any of these files, and skip downloading any dupes

from huggingface_hub import get_hf_file_metadata, hf_hub_url

import sys

# Current working directory (should be project root)
project_root = Path.cwd()
sys.path.insert(0, str(project_root))

from utils.file_processing.raw_data_file_dupe_checks import FileRegistry  # noqa: E402

# Init registry
registry = FileRegistry()

# Remove files already processed
non_dupe_files = []
for f in all_file_names_in_month:
    url = hf_hub_url(
        repo_id="Lichess/standard-chess-games",
        repo_type="dataset",
        filename=f,
    )
    meta = get_hf_file_metadata(url=url)
    size = meta.size
    etag = meta.etag
    if not registry.is_file_processed(f"{year}-{month}", Path(f).name, size, etag):
        non_dupe_files.append(f)

print(len(non_dupe_files), "new files to download")

69 new files to download


## Config

Now to configure how we want our downloading and processing pipeline to operate.

In [4]:
# Config stuff
import importlib
from utils.database import db_utils
importlib.reload(db_utils)

from utils.downloading_raw_parquet_data.raw_parquet_data_file_downloader import (
    download_single_parquet_file,
)
from utils.file_processing.process_parquet_file import (
    process_parquet_file,
)
from utils.file_processing.types_and_classes import ProcessingConfig
from utils.database.db_utils import get_db_connection, setup_database


# --- Configuration ---
year, month = 2025, 3
max_files_to_download = 5  # For testing; set to None to process all new files
local_dir = Path("../data/raw/better_downloading_experiments")
local_dir.mkdir(parents=True, exist_ok=True)

# Define the path for the DuckDB database file.
db_path = Path("../data/processed/chess_games.db")
db_path.parent.mkdir(parents=True, exist_ok=True)


# Base config for processing. This will be used for each file.
base_config = ProcessingConfig(
    parquet_path="",  # This will be set per-file
    db_path=db_path,
    batch_size=400_000,
    min_player_rating=1200,
    max_elo_difference_between_players=100,
    allowed_time_controls={"Blitz", "Rapid", "Classical"},
)

# --- Database Initialization ---
# Set up the database schema before starting any processing.
# This is idempotent; it will only create tables if they don't exist.
with get_db_connection(db_path) as con:
    setup_database(con)


# --- File Selection ---
# Use the list of non-duplicate files from the previous cell
files_to_download = non_dupe_files
if max_files_to_download is not None:
    files_to_download = non_dupe_files[:max_files_to_download]

print(f"Prepared to download and process {len(files_to_download)} files.")

Initializing database schema...
Database tables are ready.
Prepared to download and process 5 files.


## Downloading, processing, deleting

Now, we'lll download, process and delete our raw data files one by one.

The workflow is:

1. Download a file
2. Process that file, extracting the game data we want
3. Delete that file

In [5]:
import time
import os
# --- Main Loop ---
total_start_time = time.time()

for i, file_to_download in enumerate(files_to_download):
    print(f"\n--- Downloading [{i+1}/{len(files_to_download)}] ---")

    # 1. Download the file
    downloaded_file_path = download_single_parquet_file(
        repo_id="Lichess/standard-chess-games",
        repo_type="dataset",
        file_to_download=file_to_download,
        local_dir=local_dir,
        year=year,
        month=month,
    )

    if not downloaded_file_path:
        print(f"DOWNLOAD FAILED for {file_to_download}. Skipping.")
        continue

    print(f"Successfully downloaded: {downloaded_file_path.name}")

    # Get metadata for the downloaded file
    url = hf_hub_url(
        repo_id="Lichess/standard-chess-games",
        repo_type="dataset",
        filename=file_to_download,
    )
    meta = get_hf_file_metadata(url=url)

    # 2. Process the file
    # Create a specific config for this file by updating the path.
    file_config = base_config.replace(parquet_path=str(downloaded_file_path))

    # The file_context dictionary provides logging/ETA info for the processing function.
    file_context = {
        "current_file_num": i + 1,
        "total_files": len(files_to_download),
        "total_start_time": total_start_time,
    }

    # The main processing call. All data is now written to the DuckDB file.
    is_processing_successful = process_parquet_file(
        config=file_config,
        file_context=file_context,
    )

    # 3. Register and Delete on Success
    if is_processing_successful:
        print(f"PROCESSING SUCCESSFUL for {downloaded_file_path.name}")
        # Register the file as processed so we don't re-download it.
        registry.mark_file_processed(
            month=f"{year}-{month}",
            filename=downloaded_file_path.name,
            size=meta.size,
            etag=meta.etag,
        )
        print("Registered file as processed.")
        # Delete the local raw file to save space.
        os.remove(downloaded_file_path)
        print(f"Deleted local file: {downloaded_file_path.name}")
    else:
        print(f"PROCESSING FAILED for {downloaded_file_path.name}")
        # As per the plan, we mark the file as processed even on failure
        # to avoid getting stuck on a problematic file.
        registry.mark_file_processed(
            month=f"{year}-{month}",
            filename=downloaded_file_path.name,
            size=meta.size,
            etag=meta.etag,
        )
        print("Registered file as processed to avoid re-downloading.")
        os.remove(downloaded_file_path)
        print(f"Deleted local file: {downloaded_file_path.name}")



--- Downloading [1/5] ---
File saved to ../data/raw/better_downloading_experiments/2025-03-train-00000-of-00069.parquet
Successfully downloaded: 2025-03-train-00000-of-00069.parquet
Processing 1,413,223 rows in 4 batches of size 400,000

--- Starting Batch 1/4 ---
    _get_or_create_ids for player with 97386 names.
    All 97386 player names already exist in DB.
    _get_or_create_ids for opening with 457 names.
    Found 448 new opening names to insert: ['E80', 'B51', 'C39', 'A10', 'C46', 'D71', 'C11', 'D48', 'A53', 'E00']...
An error occurred during file processing: Constraint Error: NOT NULL constraint failed: opening.eco
Database connection closed.
PROCESSING FAILED for 2025-03-train-00000-of-00069.parquet
Registered file as processed to avoid re-downloading.
Deleted local file: 2025-03-train-00000-of-00069.parquet

--- Downloading [2/5] ---
File saved to ../data/raw/better_downloading_experiments/2025-03-train-00001-of-00069.parquet
Successfully downloaded: 2025-03-train-00001-of

KeyboardInterrupt: 